In [215]:
from typing import Tuple
import re
import xml.etree.ElementTree as etree
from collections import defaultdict, OrderedDict
from functools import reduce

In [3]:
import numpy as np
import pandas as pd

## Read wikipedia dump

In [6]:
root = etree.parse('../data/kowiki.xml').getroot()

In [109]:
prefix = '{http://www.mediawiki.org/xml/export-0.10/}'
keywords = ['title', 'id', 'revision/text']

In [132]:
context = defaultdict(dict)

In [133]:
for page in root[1:]:
    data = {keyword: reduce(lambda node, key: node.find(f'{prefix}{key}'), [page, *keyword.split('/')]).text for keyword in keywords}
    context[data['title']] = data

## Parser

In [376]:
regex_document = re.compile('\[\[(.[^:]+?)\]\]')
regex_table = re.compile('\{\{((.|\n)*)\}\}')
regex_table_column = re.compile('\|(.+?)\=([^\|]*)')
regex_redirect = re.compile('#넘겨주기 \[\[(.+?)\]\]')

regex_tags = OrderedDict({
    'regex_del': (r'\1', re.compile('~~(.+?)~~')),
    'regex_dash': (r'\1', re.compile('--(.+?)--')),
    'regex_link': (r'\1', re.compile('\[\[(.[^:\[\]]+?)\|(.[^:\[\]]+?)\]\]')),
    'regex_attach': ('attach', re.compile('\[\[파일:(.+?)\]\]')),
    'regex_document': (r'\1', re.compile('\[\[(.[^:]+?)\]\]')),
    'regex_tag': ('', re.compile('\<(.+?)\>')),
    'regex_comment': ('', re.compile('\[\*(.+?)\]')),
})

In [377]:
def parse(text: str) -> str:
    def _parse(text: str, target: str, tag: re.Pattern) -> Tuple[str, int]:
        return tag.subn(target, text)
    
    count = 0
    
    while True:
        for target_tag in regex_tags.values():
            text, count = _parse(text, *target_tag)
            if count:
                break
        if not count:
            break
            
    return text.strip()

## Get drama titles from lists

In [378]:
interested = ["대한민국의 텔레비전 드라마 목록/ㄱ", "대한민국의 텔레비전 드라마 목록/ㄴ", "대한민국의 텔레비전 드라마 목록/ㄷ", "대한민국의 텔레비전 드라마 목록/ㄹ", "대한민국의 텔레비전 드라마 목록/ㅁ", "대한민국의 텔레비전 드라마 목록/ㅂ", "대한민국의 텔레비전 드라마 목록/ㅅ", "대한민국의 텔레비전 드라마 목록/ㅇ", "대한민국의 텔레비전 드라마 목록/ㅈ", "대한민국의 텔레비전 드라마 목록/ㅊ", "대한민국의 텔레비전 드라마 목록/ㅋ", "대한민국의 텔레비전 드라마 목록/ㅌ", "대한민국의 텔레비전 드라마 목록/ㅍ", "대한민국의 텔레비전 드라마 목록/ㅎ", "대한민국의 텔레비전 드라마 목록/알파벳", "대한민국의 텔레비전 드라마 목록/숫자", ]

In [379]:
titles = set()

In [380]:
for inter in interested:
    document = context[inter]['revision/text']
    matches = regex_documnet.findall(document)
    
    for match in matches:
        name, *_ = match.split('|')
        name, *_ = name.split('#')
        if name != '대한민국의 텔레비전 드라마 목록':
            titles.add(name)

## Get metadata from document

In [381]:
columns = {
    '방송 기간': ['방송기간', '방송 기간'],
    '방송 횟수': ['횟수', '방송 횟수'],
    '장르': ['장르'],
    '채널': ['채널', '방송사', '방송 채널', '방송채널'],
    '제작사': ['제작사', '제작자', '제작'],
    '극본': ['극본', '대본', '각본'],
    '출연자': ['출연자', '출연', '출연진'],
}

In [407]:
data = defaultdict(dict)
notexists = []
notparser = []

In [408]:
for title in titles:
    try:
        document, redirect = context[title]['revision/text'], True
        while redirect:
            document, redirect = regex_redirect.subn(r'\1', document)
            if redirect:
                document = context[document]['revision/text']
    except KeyError:
        notexists.append(title)
    
    d = defaultdict(str)    
    for table, *_ in regex_table.findall(document):
        for key, value in regex_table_column.findall(table):
            key = next((ckey for ckey, cvalues in columns.items() if any(cvalue in key for cvalue in cvalues)), False)
            if key:
                key = key.strip()
                d[key] = f'{d[key]} {value}'
    
    if not d:
        notparser.append(title)
    else:
        data[title] = d

In [409]:
print(f'{len(notparser)} pages are not parsable')
print(f'{len(notexists)} pages are not exists')

17 pages are not parsable
15 pages are not exists


## Create table from data

In [410]:
table_columns = list(columns.keys())
table = np.empty((0, len(table_columns) + 1))

for title, values in data.items():
    table = np.vstack((table, np.array([
        title, *tuple(map(parse, map(lambda c: values[c], table_columns)))
    ])))

In [411]:
df = pd.DataFrame(table)
df.columns = ['title', *table_columns]

In [412]:
print(df.shape)
df.head()

(1756, 8)


,title,방송 기간,방송 횟수,장르,채널,제작사,극본,출연자
0,그 여름의 태풍,2005년 5월 28일 ~ 2005년 9월 4일,30부작,드라마,SBS TV,이관희 프로덕션 김호웅,최성실,[[정다빈 (1980년)
1,좋은사람 (2016년 드라마),2016년 5월 2일 ~ 2016년 10월 28일,122부작,드라마,[[MBC TV,이민수 MBC C&I,"은주영, 최연걸(26회 ~ 122회)","우희진, 현우성 외"
2,미워도 다시 한 번 2009,2009년 2월 4일 ~ 2009년 4월 23일,24부작,멜로,KBS 2TV,오성민,조희,[[최명길 (배우)
3,백수탈출,2003년 4월 26일 ~ 2003년 8월 24일,36부작,[[텔레비전 드라마,[[SBS TV,,한준영,"이정진, [[김현수 (1978년)"
4,천사의 유혹,2009년 10월 12일 ~ 2009년 12월 22일,21부작,[[텔레비전 드라마,[[SBS (대한민국의 방송사),삼화네트웍스,[[김순옥 (작가),"배수빈, 한상진, [[이소연 (배우)"


In [413]:
df.to_csv('../results/wikipedia.csv', index=None)